In [ ]:
# --- 1. Instalación de Bibliotecas y Configuración ---
print("📦 Instalando bibliotecas necesarias...")
!pip install streamlit pyngrok boto3
print("📦 Instalando herramientas de Node.js...")
!npm install -g kill-port

from pyngrok import ngrok
import os
import time
import socket
import subprocess
import threading
from google.colab import userdata

# --- 2. Carga y Configuración de Credenciales de AWS ---
print("🔐 Cargando credenciales de AWS desde el gestor de secretos...")
try:
    os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('AWS_ACCESS_KEY_ID')
    os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('AWS_SECRET_ACCESS_KEY')
    os.environ['AWS_REGION'] = userdata.get('AWS_REGION')
    print("✅ Credenciales cargadas correctamente.")
except userdata.SecretNotFoundError as e:
    print(f"❌ Error: La credencial '{e.name}' no se encontró en los secretos de Colab.")
    print("Por favor, añádela en el panel lateral de secretos y vuelve a intentarlo.")
    exit()
except Exception as e:
    print(f"❌ Error al cargar las credenciales de AWS: {e}")
    print("Asegúrate de que las credenciales son correctas y de que el gestor de secretos de Colab tiene acceso a ellas.")
    exit()

print("🔐 Configurando ngrok...")

ngrok.set_auth_token("316IsTxL0i5JFZHXhbOyGKCuGSm_3MajSpCQMzaU1mbrYtRXX")

# --- 3. Creación del Archivo `app.py` ---
print("📄 Creando el archivo 'app.py'...")
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import boto3
import json
import os
import base64

# --- Funciones de IAGen con Bedrock ---
def generar_descripcion_producto(producto_nombre, ingredientes, beneficios):
    try:
        # Las credenciales ya están en las variables de entorno
        bedrock_runtime = boto3.client(
            service_name='bedrock-runtime',
            region_name=os.environ['AWS_REGION']
        )
        prompt = f'''
        \\n\\nHuman: Eres un experto en marketing de alimentos saludables.
        Genera una descripción de producto atractiva y persuasiva para el siguiente snack.
        ---
        Nombre del producto: {producto_nombre}
        Ingredientes clave: {ingredientes}
        Beneficios: {beneficios}
        ---
        Descripción:
        \\n\\nAssistant:
        '''
        body = json.dumps({
            "prompt": prompt,
            "max_tokens_to_sample": 200,
            "temperature": 0.7,
        })
        response = bedrock_runtime.invoke_model(
            body=body,
            modelId='anthropic.claude-v2',
            accept='application/json',
            contentType='application/json'
        )
        response_body = json.loads(response.get('body').read())
        return response_body.get('completion')
    except Exception as e:
        return f"Error al generar la descripción: {e}"

def generar_imagen_promocional(prompt_imagen):
    try:
        # Las credenciales ya están en las variables de entorno
        bedrock_runtime = boto3.client(
            service_name='bedrock-runtime',
            region_name=os.environ['AWS_REGION']
        )
        prompt = f'''
        Generate a high-quality, professional marketing image for a healthy snack.
        The image should be visually appealing and focus on the prompt:
        '{prompt_imagen}'
        '''
        body = json.dumps({
            "text_prompts": [{"text": prompt}],
            "cfg_scale": 10,
            "seed": 0,
            "steps": 50,
        })
        response = bedrock_runtime.invoke_model(
            body=body,
            modelId='stability.stable-diffusion-xl-v1',
            accept='application/json',
            contentType='application/json'
        )
        response_body = json.loads(response.get('body').read())
        image_base64 = response_body.get('artifacts')[0].get('base64')
        return f"data:image/png;base64,{image_base64}"
    except Exception as e:
        return f"Error al generar la imagen: {e}"

def generar_resumen_comentarios(comentarios):
    try:
        bedrock_runtime = boto3.client(
            service_name='bedrock-runtime',
            region_name=os.environ['AWS_REGION']
        )
        prompt = f'''
        \\n\\nHuman: Actúa como un analista de mercado experto en productos de consumo masivo. Lee los siguientes comentarios de clientes sobre un nuevo snack y genera un resumen conciso que destaque las opiniones clave, tanto positivas como negativas.

        --- Comentarios ---
        {comentarios}
        ---

        Resumen:
        \\n\\nAssistant:
        '''
        body = json.dumps({
            "prompt": prompt,
            "max_tokens_to_sample": 500,
            "temperature": 0.5,
        })
        response = bedrock_runtime.invoke_model(
            body=body,
            modelId='anthropic.claude-v2',
            accept='application/json',
            contentType='application/json'
        )
        response_body = json.loads(response.get('body').read())
        return response_body.get('completion')
    except Exception as e:
        return f"Error al generar el resumen: {e}"

# --- Interfaz de Streamlit ---
st.title("Snack Marketing AI 🤖")
st.markdown("Genera descripciones e imágenes de productos con Bedrock.")

tab1, tab2, tab3 = st.tabs(["Generador de Descripción", "Generador de Imágenes","Análisis de Comentarios de Clientes"])

with tab1:
    st.header("✍️ Generar Descripción de Producto")
    with st.form("form_descripcion"):
        nombre = st.text_input("Nombre del Producto", "Power Crunch")
        ingredientes = st.text_input("Ingredientes Clave", "Avena, almendras, chocolate negro")
        beneficios = st.text_area("Beneficios del Producto", "Alto en fibra, energía sostenida, sin azúcares añadidos")
        submitted_desc = st.form_submit_button("Generar Descripción")
        if submitted_desc:
            with st.spinner('Generando descripción...'):
                descripcion = generar_descripcion_producto(nombre, ingredientes, beneficios)
                st.subheader("Descripción Generada:")
                st.success(descripcion)

with tab2:
    st.header("🖼️ Generar Imagen Promocional")
    with st.form("form_imagen"):
        prompt_imagen = st.text_area("Prompt para la Imagen",
                                     "Foto de un bowl de snack de avena y almendras, con chocolate derretido, en un entorno rústico de madera, con luz natural. Estilo fotográfico: minimalista y saludable.")
        submitted_img = st.form_submit_button("Generar Imagen")
        if submitted_img:
            with st.spinner('Generando imagen...'):
                imagen_base64 = generar_imagen_promocional(prompt_imagen)
                st.subheader("Imagen Generada:")
                if imagen_base64.startswith("Error"):
                    st.error(imagen_base64)
                else:
                    st.image(imagen_base64, caption="Imagen promocional generada", use_column_width=True)

with tab3:
    st.header("🖼️ nálisis de Comentarios de Clientes ")
    with st.form("form_comentarios"):
        comentarios = st.text_area(
            "Pega aquí los comentarios de usuarios",
            "¡Me encanta el sabor a coco, es delicioso y crujiente!\\n"
            "El empaque es muy bonito y práctico, pero el precio es un poco alto.\\n"
            "Ideal para una merienda rápida y saludable.\\n"
            "No me gustó que es muy dulce, esperaba algo más natural."
        )
        submitted = st.form_submit_button("Generar Resumen")

    if submitted:
        if not comentarios:
            st.warning("Por favor, ingresa al menos un comentario.")
        else:
            with st.spinner('Analizando y generando resumen...'):
                resumen = generar_resumen_comentarios(comentarios)
                st.subheader("Resumen Generado:")
                st.success(resumen)
""")

# --- 4. Iniciar la Aplicación Streamlit con ngrok ---
def run_streamlit():
    """Ejecuta Streamlit en un proceso de fondo y lo mantiene activo."""
    # Prepara el diccionario de variables de entorno, incluyendo las de AWS
    env_vars = os.environ.copy()

    # Inicia Streamlit con las variables de entorno
    p = subprocess.Popen(["streamlit", "run", "app.py"], env=env_vars)
    print(f"Streamlit PID: {p.pid}")
    p.wait()

print("🚀 Iniciando la aplicación Streamlit...")
threading.Thread(target=run_streamlit, daemon=True).start()

# Bucle para comprobar si el puerto 8501 está abierto antes de conectar ngrok.
print("⏳ Comprobando si Streamlit está listo en el puerto 8501...")
port_open = False
max_retries = 30  # Máximo de 30 segundos de espera
retry_count = 0
while not port_open and retry_count < max_retries:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        if s.connect_ex(('localhost', 8501)) == 0:
            port_open = True
            print("✅ Streamlit está listo. Conectando ngrok...")
        else:
            retry_count += 1
            time.sleep(1)

if not port_open:
    print("❌ Streamlit no se inició en el puerto 8501 después de 30 segundos.")
    print("Por favor, revisa los logs de Streamlit para encontrar el error.")
    exit()

# Crea el túnel ngrok al puerto 8501.
try:
    public_url = ngrok.connect(8501)
    if public_url:
        print("\n🎉 ¡Listo! Usa esta URL para acceder a tu app de Streamlit:")
        print(f"URL pública: {public_url}")
    else:
        print("\n❌ Error: No se pudo crear el túnel de ngrok.")
        print("Asegúrate de que no tienes túneles de ngrok abiertos.")
except Exception as e:
    print(f"❌ Error al iniciar ngrok: {e}")
    print("Por favor, revisa que tu token de ngrok es correcto y que no tienes otros túneles abiertos.")